In [2]:
import numpy as np
import pandas as pd

In [ ]:
all_matches = pd.read_csv(r'/home/chetan/Downloads/ipl_male_csv2/all_matches.csv')

In [ ]:
all_matches.tail()

# Making Some Minor Changes and removing redundancies

In [6]:
all_matches_corrected = all_matches.replace(to_replace='Kings XI Punjab',value='Punjab Kings')
all_matches_corrected.replace(to_replace='Feroz Shah Kotla',value='Arun Jaitley Stadium',inplace=True)
all_matches_corrected.replace(to_replace='Sardar Patel Stadium, Motera',value='Narendra Modi Stadium',inplace=True)
all_matches_corrected.replace(to_replace='Wankhede Stadium, Mumbai',value='Wankhede Stadium',inplace=True)
all_matches_corrected.replace(to_replace=['MA Chidambaram Stadium, Chepauk','MA Chidambaram Stadium, Chepauk, Chennai']
                              ,value='MA Chidambaram Stadium',inplace=True)
all_matches_corrected.replace(to_replace='M.Chinnaswamy Stadium',value='M Chinnaswamy Stadium',inplace=True)

all_matches_corrected.replace(to_replace='Brabourne Stadium, Mumbai',value='Brabourne Stadium',inplace=True)
all_matches_corrected.replace(to_replace=['Punjab Cricket Association IS Bindra Stadium, Mohali'
                                                        ,'Punjab Cricket Association IS Bindra Stadium']
                                                        ,value='Punjab Cricket Association Stadium, Mohali',inplace=True)

all_matches_corrected.replace(to_replace='Rajiv Gandhi International Stadium, Uppal',value='Rajiv Gandhi International Stadium',inplace=True)

In [7]:
all_matches_corrected.tail()

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
198500,1254077,2021,2021-04-25,MA Chidambaram Stadium,4,0.2,Delhi Capitals,Sunrisers Hyderabad,S Dhawan,RR Pant,...,1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
198501,1254077,2021,2021-04-25,MA Chidambaram Stadium,4,0.3,Delhi Capitals,Sunrisers Hyderabad,RR Pant,S Dhawan,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198502,1254077,2021,2021-04-25,MA Chidambaram Stadium,4,0.4,Delhi Capitals,Sunrisers Hyderabad,RR Pant,S Dhawan,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198503,1254077,2021,2021-04-25,MA Chidambaram Stadium,4,0.5,Delhi Capitals,Sunrisers Hyderabad,RR Pant,S Dhawan,...,1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
198504,1254077,2021,2021-04-25,MA Chidambaram Stadium,4,0.6,Delhi Capitals,Sunrisers Hyderabad,S Dhawan,RR Pant,...,1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [8]:
all_matches = all_matches_corrected.copy() #just using the same name again for subsequent code

# Preparing the Statistics of Batsmen and Bowlers from Past Data

In [9]:
def get_batsmen_statistics(matches_df):

    batsmen_df = pd.DataFrame(columns = ['Batsman','Runs','Balls_Faced','Fours','Sixes','Strike_Rate'])
    strikers = matches_df.striker.unique()
    for striker in strikers:    
        batsman_ball2ball = matches_df[matches_df.striker==striker].runs_off_bat
        runs = batsman_ball2ball.sum()
        fours = int(batsman_ball2ball[batsman_ball2ball==4].count())
        sixes = int(batsman_ball2ball[batsman_ball2ball==6].count())
        balls = int(batsman_ball2ball.shape[0])
        strike_rate = round(runs/balls*100,2)
        batsmen_df=batsmen_df.append({'Batsman':striker,
                                      'Runs':runs,
                                      'Balls_Faced':balls,
                                      'Fours':fours,
                                      'Sixes':sixes,
                                      'Strike_Rate':strike_rate},ignore_index=True)
    
    return batsmen_df

def get_bowlers_statistics(matches_df):
    
    matches_df = matches_df.iloc[:]
    matches_df.fillna(0,inplace=True)
    bowlers_df = pd.DataFrame(columns = ['Bowler','Runs_Conceded','Balls','Economy_Rate'])
    bowlers = matches_df.bowler.unique()
    matches_df['total_runs'] = matches_df['runs_off_bat']+matches_df['wides']+matches_df['noballs']
    
    for bowler in bowlers:
        bowler_ball2ball = matches_df[matches_df.bowler==bowler]
        runs_conceded = bowler_ball2ball.total_runs.sum()

        #calculates the total number of wide balls and noballs
        wide_balls = bowler_ball2ball[bowler_ball2ball.wides !=0].shape[0]
        noballs = bowler_ball2ball[bowler_ball2ball.noballs !=0].shape[0]
        extra_balls = wide_balls+noballs
        
        #calculates the total number of overs
        balls = bowler_ball2ball.shape[0]
        overs = (balls-extra_balls)/6
        
        #economy rate
        economy_rate = round(runs_conceded/overs,2)
        
        bowlers_df = bowlers_df.append({'Bowler':bowler,'Economy_Rate':economy_rate,
                                        'Runs_Conceded':runs_conceded,'Balls':balls-extra_balls},ignore_index = True)
        
    return bowlers_df

In [10]:
all_matches.fillna(0,inplace=True)

#PP stands for Powerplay
all_matches_PP = all_matches[all_matches.ball<5.99]

In [11]:
batsmen_database_PP = get_batsmen_statistics(all_matches_PP)
batsmen_database_PP

,Batsman,Runs,Balls_Faced,Fours,Sixes,Strike_Rate
0,SC Ganguly,529,540,76,9,97.96
1,BB McCullum,1557,1278,184,70,121.83
2,RT Ponting,45,75,2,1,60.00
3,R Dravid,905,855,148,7,105.85
4,W Jaffer,77,95,9,1,81.05
...,...,...,...,...,...,...
288,RM Patidar,9,11,1,0,81.82
289,Shahbaz Ahmed,14,9,0,1,155.56
290,S Dube,12,16,1,1,75.00
291,M Shahrukh Khan,0,2,0,0,0.00


In [12]:
bowlers_database_PP = get_bowlers_statistics(all_matches_PP)
bowlers_database_PP

,Bowler,Runs_Conceded,Balls,Economy_Rate
0,P Kumar,1754.0,1572,6.69
1,Z Khan,1452.0,1292,6.74
2,AA Noffke,22.0,6,22.00
3,AB Dinda,991.0,810,7.34
4,I Sharma,1444.0,1215,7.13
...,...,...,...,...
348,RP Meredith,26.0,12,13.00
349,Abhishek Sharma,22.0,18,7.33
350,LI Meriwala,28.0,12,14.00
351,Jalaj S Saxena,10.0,6,10.00


In [14]:
#Running the followling lines of code with save the statistics as csv files

#batsmen_database_PP.to_csv('batsmen_database_PP.csv')
#bowlers_database_PP.to_csv('bowlers_database_PP.csv')

# Data Preprocessing

In [13]:
data_df = all_matches.copy()

#filling nan with 0
data_df.fillna(0,inplace=True)

#considering only the powerplay
data_df = data_df[data_df.ball<5.99]

#eliminating super overs
data_df = data_df[data_df.innings<=2] 

#dropping useless columns
data_df.drop(['non_striker','wides','noballs','byes','legbyes',
              'penalty','other_wicket_type','other_player_dismissed'],axis=1,inplace=True)

#adding total runs column
data_df['total_runs']= data_df['runs_off_bat']+data_df['extras']



In [14]:
data_df

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,bowler,runs_off_bat,extras,wicket_type,player_dismissed,total_runs
0,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,P Kumar,0,1,0,0,1
1,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.2,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,0,0,0,0,0
2,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.3,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,0,1,0,0,1
3,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.4,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,0,0,0,0,0
4,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.5,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198400,1254077,2021,2021-04-25,MA Chidambaram Stadium,2,5.2,Sunrisers Hyderabad,Delhi Capitals,KS Williamson,Avesh Khan,1,0,0,0,1
198401,1254077,2021,2021-04-25,MA Chidambaram Stadium,2,5.3,Sunrisers Hyderabad,Delhi Capitals,JM Bairstow,Avesh Khan,6,0,0,0,6
198402,1254077,2021,2021-04-25,MA Chidambaram Stadium,2,5.4,Sunrisers Hyderabad,Delhi Capitals,JM Bairstow,Avesh Khan,0,0,caught,JM Bairstow,0
198403,1254077,2021,2021-04-25,MA Chidambaram Stadium,2,5.5,Sunrisers Hyderabad,Delhi Capitals,Virat Singh,Avesh Khan,0,0,0,0,0


In [15]:
data_df_processed = pd.DataFrame(columns = ['match_id','season','venue','date','D/N','innings','batting_team','bowling_team','batsmen','bowlers',
                                   'avg_strike_rate','avg_economy_rate','wickets','runs'])

In [16]:
data_df_processed

,match_id,season,venue,date,D/N,innings,batting_team,bowling_team,batsmen,bowlers,avg_strike_rate,avg_economy_rate,wickets,runs


In [17]:
#returns the avg economy rate of the list of bowlers passed as an argument

def get_avg_economy_rate(bowlers):
    
    if len(bowlers)==0:
        return 7.0
    
    economy_t = 0
    for bowler in bowlers:
        if bowler in bowlers_database_PP.Bowler.values:
            index_of_bowler = bowlers_database_PP[bowlers_database_PP.Bowler==bowler].index.values
            economy = float(bowlers_database_PP.Economy_Rate.iloc[index_of_bowler])
        else:
            economy = 7.0
           
        economy_t += economy
    return round(economy_t/len(bowlers),2)

#***************************************************
#function returns the average strike rate of the list of batsmen passed as an argument

def get_avg_strike_rate(batsmen):
    
    if len(batsmen)==0:
        return 100
    strike_rate_t=0  #stores the total strike rate
    for batsman in batsmen:
        if batsman in batsmen_database_PP.Batsman.values:
            index_of_batsman = batsmen_database_PP[batsmen_database_PP.Batsman==batsman].index.values
            strike_rate = float(batsmen_database_PP.Strike_Rate.iloc[index_of_batsman])
            
        else:    #if the batsman is new, then assuming his strikerate is 100
            strike_rate = 100
        
        strike_rate_t += strike_rate
            
    return round(strike_rate_t/len(batsmen),2)

In [18]:
match_ids = data_df.match_id.unique()

for match_id in match_ids:
    
    match_df = data_df[data_df.match_id==match_id]
    match_deets = match_df.iloc[0]
    season = match_deets.season
    venue = match_deets.venue
    batting_team = match_deets.batting_team
    bowling_team = match_deets.bowling_team
    date = match_deets.start_date
    
    for i in [1,2]:
        
        if i==2:
            (batting_team,bowling_team) = (bowling_team,batting_team)
        
        innings_df = match_df[match_df.innings==i]
        runs = innings_df.total_runs.sum()
        #calculating the number of wickets
        wicket_balls = innings_df[innings_df.player_dismissed != 0]
        wickets = wicket_balls.shape[0]

        batsmen = innings_df.striker.unique()
        bowlers = innings_df.bowler.unique()

        avg_strike_rate = get_avg_strike_rate(batsmen)
        avg_economy_rate = get_avg_economy_rate(bowlers)

        row_dict = {'match_id':match_id,'season':season,'venue':venue,'date':date,
                    'innings':i,'batting_team':batting_team,
                    'bowling_team':bowling_team,'batsmen':batsmen,'bowlers':bowlers,
                    'avg_strike_rate':avg_strike_rate,'avg_economy_rate':avg_economy_rate,
                    'wickets':wickets,'runs':runs}

        
        data_df_processed=data_df_processed.append(row_dict,ignore_index=True)

In [24]:
#Adding if a match is 4pm or 8pm match

test = data_df_processed.copy()
dates = test.date.unique()
for date in dates:
    matches_on_date = test[test.date==date]
    num = len(matches_on_date)
    indices_of_matches= matches_on_date.index.values
    
    if num==2:
        test.loc[indices_of_matches,'D/N'] = '8pm'
    
    elif num==4:
        test.loc[indices_of_matches[:2],'D/N']= '4pm'
        test.loc[indices_of_matches[2:4],'D/N']= '8pm'
    
data_df_processed = test.copy()

In [25]:
data_df_processed

,match_id,season,venue,date,D/N,innings,batting_team,bowling_team,batsmen,bowlers,avg_strike_rate,avg_economy_rate,wickets,runs
0,335982,2007/08,M Chinnaswamy Stadium,2008-04-18,8pm,1,Kolkata Knight Riders,Royal Challengers Bangalore,['SC Ganguly' 'BB McCullum' 'RT Ponting'],['P Kumar' 'Z Khan' 'AA Noffke'],93.26,11.81,1,61
1,335982,2007/08,M Chinnaswamy Stadium,2008-04-18,8pm,2,Royal Challengers Bangalore,Kolkata Knight Riders,['R Dravid' 'W Jaffer' 'V Kohli' 'JH Kallis' '...,['AB Dinda' 'I Sharma' 'AB Agarkar'],96.93,7.99,4,26
2,335983,2007/08,"Punjab Cricket Association Stadium, Mohali",2008-04-19,4pm,1,Chennai Super Kings,Punjab Kings,['PA Patel' 'ML Hayden' 'MEK Hussey'],['B Lee' 'S Sreesanth' 'JR Hopes'],115.25,7.64,1,53
3,335983,2007/08,"Punjab Cricket Association Stadium, Mohali",2008-04-19,4pm,2,Punjab Kings,Chennai Super Kings,['K Goel' 'JR Hopes'],['JDP Oram' 'MS Gony'],108.70,9.14,1,63
4,335984,2007/08,Arun Jaitley Stadium,2008-04-19,8pm,1,Rajasthan Royals,Delhi Daredevils,['T Kohli' 'YK Pathan' 'SR Watson' 'M Kaif'],['GD McGrath' 'B Geeves' 'MF Maharoof'],90.31,8.54,2,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1667,1254075,2021,Wankhede Stadium,2021-04-24,8pm,2,Rajasthan Royals,Kolkata Knight Riders,['JC Buttler' 'YBK Jaiswal' 'SV Samson' 'S Dube'],['Shivam Mavi' 'PJ Cummins' 'CV Varun' 'SP Nar...,106.12,7.65,2,50
1668,1254076,2021,Wankhede Stadium,2021-04-25,4pm,1,Chennai Super Kings,Royal Challengers Bangalore,['RD Gaikwad' 'F du Plessis'],['Mohammed Siraj' 'KA Jamieson' 'YS Chahal' 'N...,110.68,8.15,0,51
1669,1254076,2021,Wankhede Stadium,2021-04-25,4pm,2,Royal Challengers Bangalore,Chennai Super Kings,['V Kohli' 'D Padikkal' 'Washington Sundar' 'G...,['DL Chahar' 'SM Curran' 'SN Thakur'],107.79,7.98,2,65
1670,1254077,2021,MA Chidambaram Stadium,2021-04-25,8pm,1,Delhi Capitals,Sunrisers Hyderabad,['PP Shaw' 'S Dhawan'],['KK Ahmed' 'Abhishek Sharma' 'S Kaul' 'J Such...,128.00,8.33,0,51


In [26]:
#Run the following line of code to save the processed data

#data_df_processed.to_csv('data_df_processed.csv',index=False)

# Preparing the Data for the Model

In [27]:
data_df_processed = pd.read_csv('data_df_processed.csv')
data_df_processed

,match_id,season,venue,date,D/N,innings,batting_team,bowling_team,batsmen,bowlers,avg_strike_rate,avg_economy_rate,wickets,runs
0,335982,2007/08,M Chinnaswamy Stadium,2008-04-18,8pm,1,Kolkata Knight Riders,Royal Challengers Bangalore,['SC Ganguly' 'BB McCullum' 'RT Ponting'],['P Kumar' 'Z Khan' 'AA Noffke'],93.26,11.81,1,61
1,335982,2007/08,M Chinnaswamy Stadium,2008-04-18,8pm,2,Royal Challengers Bangalore,Kolkata Knight Riders,['R Dravid' 'W Jaffer' 'V Kohli' 'JH Kallis' '...,['AB Dinda' 'I Sharma' 'AB Agarkar'],96.93,7.99,4,26
2,335983,2007/08,"Punjab Cricket Association Stadium, Mohali",2008-04-19,4pm,1,Chennai Super Kings,Punjab Kings,['PA Patel' 'ML Hayden' 'MEK Hussey'],['B Lee' 'S Sreesanth' 'JR Hopes'],115.25,7.64,1,53
3,335983,2007/08,"Punjab Cricket Association Stadium, Mohali",2008-04-19,4pm,2,Punjab Kings,Chennai Super Kings,['K Goel' 'JR Hopes'],['JDP Oram' 'MS Gony'],108.70,9.14,1,63
4,335984,2007/08,Arun Jaitley Stadium,2008-04-19,8pm,1,Rajasthan Royals,Delhi Daredevils,['T Kohli' 'YK Pathan' 'SR Watson' 'M Kaif'],['GD McGrath' 'B Geeves' 'MF Maharoof'],90.31,8.54,2,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1667,1254075,2021,Wankhede Stadium,2021-04-24,8pm,2,Rajasthan Royals,Kolkata Knight Riders,['JC Buttler' 'YBK Jaiswal' 'SV Samson' 'S Dube'],['Shivam Mavi' 'PJ Cummins' 'CV Varun' 'SP Nar...,106.12,7.65,2,50
1668,1254076,2021,Wankhede Stadium,2021-04-25,4pm,1,Chennai Super Kings,Royal Challengers Bangalore,['RD Gaikwad' 'F du Plessis'],['Mohammed Siraj' 'KA Jamieson' 'YS Chahal' 'N...,110.68,8.15,0,51
1669,1254076,2021,Wankhede Stadium,2021-04-25,4pm,2,Royal Challengers Bangalore,Chennai Super Kings,['V Kohli' 'D Padikkal' 'Washington Sundar' 'G...,['DL Chahar' 'SM Curran' 'SN Thakur'],107.79,7.98,2,65
1670,1254077,2021,MA Chidambaram Stadium,2021-04-25,8pm,1,Delhi Capitals,Sunrisers Hyderabad,['PP Shaw' 'S Dhawan'],['KK Ahmed' 'Abhishek Sharma' 'S Kaul' 'J Such...,128.00,8.33,0,51


In [28]:
#label encoding the venue and teams
#RUN THIS BLOCK OF CODE ONLY ONCE
from sklearn.preprocessing import LabelEncoder,MinMaxScaler

data_df_final = data_df_processed.copy()
venue_encoder = LabelEncoder()
team_encoder = LabelEncoder()

data_df_final['venue'] = venue_encoder.fit_transform(data_df_final.venue)

team = team_encoder.fit(data_df_final.batting_team)
data_df_final['batting_team'] = team.transform(data_df_final.batting_team)
data_df_final['bowling_team'] = team.transform(data_df_final.bowling_team)

data_df_final.head()

,match_id,season,venue,date,D/N,innings,batting_team,bowling_team,batsmen,bowlers,avg_strike_rate,avg_economy_rate,wickets,runs
0,335982,2007/08,14,2008-04-18,8pm,1,6,13,['SC Ganguly' 'BB McCullum' 'RT Ponting'],['P Kumar' 'Z Khan' 'AA Noffke'],93.26,11.81,1,61
1,335982,2007/08,14,2008-04-18,8pm,2,13,6,['R Dravid' 'W Jaffer' 'V Kohli' 'JH Kallis' '...,['AB Dinda' 'I Sharma' 'AB Agarkar'],96.93,7.99,4,26
2,335983,2007/08,22,2008-04-19,4pm,1,0,9,['PA Patel' 'ML Hayden' 'MEK Hussey'],['B Lee' 'S Sreesanth' 'JR Hopes'],115.25,7.64,1,53
3,335983,2007/08,22,2008-04-19,4pm,2,9,0,['K Goel' 'JR Hopes'],['JDP Oram' 'MS Gony'],108.70,9.14,1,63
4,335984,2007/08,0,2008-04-19,8pm,1,10,3,['T Kohli' 'YK Pathan' 'SR Watson' 'M Kaif'],['GD McGrath' 'B Geeves' 'MF Maharoof'],90.31,8.54,2,40


In [29]:
chosen_features = ['venue','innings','batting_team','bowling_team','avg_strike_rate',
                   'avg_economy_rate','wickets']

#converting the dataframe into a numpy array
X = data_df_final[chosen_features].values
y = data_df_final.runs.values

#making the data type compatible for model training
X=X.astype(np.float32)
y=y.astype(np.float32)

print(X.shape)
print(y.shape)

(1672, 7)
(1672,)


In [30]:
#scaling the data

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Model Training

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

In [32]:
X_train,X_test,y_train,y_test = train_test_split(X_scaled,y)

In [49]:
#Accuracy metric functions

def custom_accuracy_metric(y_true,y_pred,threshhold = 3):
    true_mat = y_pred[np.abs(y_true-y_pred)<=threshhold]
    return len(true_mat)/len(y_pred)

def analyse_model(model,X_train,X_test,y_train,y_test,threshhold=3):
    
    y_pred_train = np.array(list(map(int,model.predict(X_train))))
    y_pred_test = np.array(list(map(int,model.predict(X_test))))
    
    print('R2 Score for training set:',round(r2_score(y_train,y_pred_train),3))
    print('R2 Score for test set:',round(r2_score(y_test,y_pred_test),3))
    print()
    
    #using custom accuracy metric
    print('Accuracy for a threshhold of {} runs'.format(threshhold))
    print('Training set: ',round(custom_accuracy_metric(y_train,y_pred_train,threshhold)*100,2),'%')
    print('Test set: ',round(custom_accuracy_metric(y_test,y_pred_test,threshhold)*100,2),'%')

Linear Regression Model

In [53]:
#lr stands for linear regression
lr = LinearRegression()
lr.fit(X_train,y_train)

print('**** Linear Regression ****')
analyse_model(lr,X_train,X_test,y_train,y_test,5)

**** Linear Regression ****
R2 Score for training set: 0.297
R2 Score for test set: 0.307

Accuracy for a threshhold of 5 runs
Training set:  43.54 %
Test set:  46.41 %


In [54]:
tree = DecisionTreeRegressor()
tree.fit(X_train,y_train)

print('**** Decision Tree Regressor ****')
analyse_model(tree,X_train,X_test,y_train,y_test,5)

**** Decision Tree Regressor ****
R2 Score for training set: 1.0
R2 Score for test set: -0.621

Accuracy for a threshhold of 5 runs
Training set:  100.0 %
Test set:  27.99 %


In [55]:
br = BayesianRidge()
br.fit(X_train,y_train)

print('**** Bayesian Ridge ****')
analyse_model(br,X_train,X_test,y_train,y_test,5)

**** Bayesian Ridge ****
R2 Score for training set: 0.297
R2 Score for test set: 0.31

Accuracy for a threshhold of 5 runs
Training set:  43.62 %
Test set:  46.41 %


# Neural Network Model

In [56]:
from tensorflow import keras
from tensorflow.keras.layers import Dense

In [60]:
DL_model = keras.Sequential()
DL_model.add(Dense(64,input_dim=7, activation= "relu"))
DL_model.add(Dense(32, activation= "relu"))
DL_model.add(Dense(1))

DL_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                512       
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 2,625
Trainable params: 2,625
Non-trainable params: 0
_________________________________________________________________


In [61]:
DL_model.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])

DL_model.fit(X_train,y_train,epochs=20)

Epoch 1/20
40/40 [==============================] - 2s 2ms/step - loss: 2259.7428 - mean_squared_error: 2259.7428
Epoch 2/20
40/40 [==============================] - 0s 2ms/step - loss: 2067.2258 - mean_squared_error: 2067.2258
Epoch 3/20
40/40 [==============================] - 0s 2ms/step - loss: 1599.0931 - mean_squared_error: 1599.0931
Epoch 4/20
40/40 [==============================] - 0s 2ms/step - loss: 728.3088 - mean_squared_error: 728.3088
Epoch 5/20
40/40 [==============================] - 0s 2ms/step - loss: 230.8749 - mean_squared_error: 230.8749
Epoch 6/20
40/40 [==============================] - 0s 2ms/step - loss: 223.1573 - mean_squared_error: 223.1573
Epoch 7/20
40/40 [==============================] - 0s 2ms/step - loss: 203.6912 - mean_squared_error: 203.6912
Epoch 8/20
40/40 [==============================] - 0s 2ms/step - loss: 185.6233 - mean_squared_error: 185.6233
Epoch 9/20
40/40 [==============================] - 0s 2ms/step - loss: 181.9027 - mean_squared_er

In [62]:
print('**** Neural Network with 1 hidden layer ****')
analyse_model(DL_model,X_train,X_test,y_train,y_test,threshhold=5)

**** Neural Network with 1 hidden layer ****
R2 Score for training set: 0.23
R2 Score for test set: 0.259

Accuracy for a threshhold of 5 runs
Training set:  42.42 %
Test set:  43.06 %


# Saving the Models

In [63]:
import joblib

In [64]:
joblib.dump(venue_encoder,'venue_encoder.joblib')
joblib.dump(team_encoder,'team_encoder.joblib')
joblib.dump(scaler,'scaler.joblib')
#DL_model.save('DL_model.h5')
joblib.dump(lr,'linear_regression.joblib')

['linear_regression.joblib']

In [65]:
joblib.dump(br,'bayesian_ridge_regression.joblib')

['bayesian_ridge_regression.joblib']